### Topic B: Estimating Parametric Models using MLE

#### b.1 Method Implementation

##### Implementation of covariance matrix calculation and discriminant for the all the three models

Covariance Matrix : 
> The covariance matrix is a key parameter in the multivariate normal distribution, which describes the relationship between the different features (or variables) in the data. It is a square matrix that contains the variances of each variable along the diagonal, and the covariances between pairs of variables off the diagonal. In other words, it captures the extent to which two variables vary together.

Discriminant Function :

> In a classification problem with multiple classes, we need to define a decision boundary that separates the different classes. Discriminant functions are a common way to define these decision boundaries. A discriminant function takes in a set of features and outputs a score for each class, based on how well the features match the distribution of that class. The class with the highest score is then chosen as the predicted class label.

> In a parametric model based on the multivariate normal distribution, the discriminant functions are typically quadratic functions of the input features. These functions are derived from Bayes' theorem and the assumption of equal class priors, and can be expressed in terms of the means and covariance matrices of the class-conditional PDFs.

![Alt text](5.17.png)

![Alt text](5.22.png)

![Alt text](5.24.png)

In [1]:
import numpy as np
import pandas as pd
import random

class IndependentCovar:
    # below function is to get covariance matrices 
    def getCovar(self, C1: pd.DataFrame, C2: pd.DataFrame,
                 P_C1: float, P_C2: float,
                 m1: np.array, m2: np.array):
        # below is the calculation for covariance matrices for class C1 & C2
        matrix1 = np.vstack([i-m1 for i in C1])
        matrix2 = np.vstack([j-m2 for j in C2])
        S1 = np.matmul(matrix1.T, matrix1)/C1.shape[0]
        S2 = np.matmul(matrix2.T, matrix2)/C2.shape[0]
        return S1, S2
    # below is to function to classify a new datapoint and here the PC1 & PC2 are prior probabilities , m1 & m2 are mean of both the classes respectively and S1 & S2 are covariance matrices which we calculated in above function 
    def classify(self, x: pd.Series, P_C1: float, P_C2: float,
                 m1: np.array, m2: np.array,
                 S1: np.array, S2: np.array):
        # this is the discriminant function eqn implementation of equation taken from 5.17 of Alpaydin & we calculated discriminant functions for both the classes 
        g1 = -0.5*(np.log(np.linalg.det(S1))
                 + (x - m1).T.dot(np.linalg.inv(S1)).dot(x - m1))\
             + np.log(P_C1)
        g2 = -0.5*(np.log(np.linalg.det(S2))
                 + (x - m2).T.dot(np.linalg.inv(S2)).dot(x - m2))\
             + np.log(P_C2)
        
        # If discriminant functions are equal, classify randomly otherwise, classify based on which discriminant function is larger
        if g1 == g2:
            return random.choice([1, 2])
        elif g1 > g2:
            return 1
        else:
            return 2
            
class EqualCovar:
    def getCovar(self, C1: pd.DataFrame, C2: pd.DataFrame,
                 P_C1: float, P_C2: float,
                 m1: np.array, m2:np.array):
        matrix1 = np.vstack([i-m1 for i in C1])
        matrix2 = np.vstack([j-m2 for j in C2])
        S1 = np.matmul(matrix1.T, matrix1)/C1.shape[0]
        S2 = np.matmul(matrix2.T, matrix2)/C2.shape[0]
        # Compute the pooled covariance matrix
        # The pooled covariance matrix is a weighted average of the covariance matrices of two or more groups or classes, where the weight of each group is proportional to the number of samples in that group. It is commonly used in statistical analysis and machine learning, particularly in classification problems where there are multiple groups or classes.
        S = P_C1*S1 + P_C2*S2
        return S, S
    
    def classify(self, x: pd.Series, P_C1: float, P_C2: float,
                 m1: np.array, m2: np.array,
                 S1: np.array, S2: np.array):
        # this is the discriminant function eqn implementation of equation taken from 5.22 of Alpaydin & we calculated discriminant functions for both the classes 
        g1 = -0.5*(x - m1).T.dot(np.linalg.inv(S1)).dot(x - m1) + np.log(P_C1)
        g2 = -0.5*(x - m2).T.dot(np.linalg.inv(S2)).dot(x - m2) + np.log(P_C2)
        if g1 == g2: return random.choice([1, 2])
        elif g1 > g2: return 1
        else: return 2

class DiagonalCovar:
    def getCovar(self, C1: pd.DataFrame, C2: pd.DataFrame,
                 P_C1: float, P_C2: float,
                 m1: np.array, m2:np.array):
        matrix1 = np.vstack([i-m1 for i in C1])
        matrix2 = np.vstack([j-m2 for j in C2])
        S1 = np.matmul(matrix1.T, matrix1)/C1.shape[0]
        S2 = np.matmul(matrix2.T, matrix2)/C2.shape[0]
        return np.diag(np.diag(S1)), np.diag(np.diag(S2))
        
    def classify(self, x: pd.Series, P_C1: float, P_C2: float,
                     m1: np.array, m2: np.array,
                     S1: np.array, S2: np.array):
        # this is the discriminant function eqn implementation of equation taken from 5.22 of Alpaydin & we calculated discriminant functions for both the classes 
        g1 = -0.5*(np.log(np.linalg.det(S1))
                 + (x - m1).T.dot(np.linalg.inv(S1)).dot(x - m1))\
             + np.log(P_C1)
        g2 = -0.5*(np.log(np.linalg.det(S2))
                 + (x - m2).T.dot(np.linalg.inv(S2)).dot(x - m2))\
             + np.log(P_C2)
        if g1 == g2: return random.choice([1, 2])
        elif g1 > g2: return 1
        else: return 2

##### Test Error function implementation

In [2]:
def testError(model, test: pd.DataFrame,
              P_C1: float, P_C2: float,
              m1: np.array, m2: np.array,
              S1: np.array, S2: np.array):
    actual_values = [i for i in test[8]]
    predicted_values = [None for _ in actual_values]
    test = test.drop(8, axis=1)
    # Iterating through each row and predicting the class using classifier
    for i in range(test.shape[0]):
        predicted_values[i] = model.classify(test.iloc[i],P_C1, P_C2, m1, m2, S1, S2)
    MSE = sum([abs(a - p) for a, p in zip(actual_values, predicted_values)])/test.shape[0]
    return MSE


##### Main Function Implementation
> The MultiGaussian() function is to implement the entire process of training and testing a multivariate
Gaussian classifier.



In [3]:
def MultiGaussian(training_data: str, testing_data: str, Model: int):
    if Model not in [1, 2, 3]:
        raise Exception("'Model'should be set to 1, 2, or 3")
        
    train_df = pd.read_csv(training_data, header=None)
    C1 = train_df.groupby(8).get_group(1).drop(8, axis=1).to_numpy()
    C2 = train_df.groupby(8).get_group(2).drop(8, axis=1).to_numpy()
    P_C1 = C1.shape[0] / train_df.shape[0]
    P_C2 = 1 - P_C1
    m1 = C1.mean(axis=0)
    m2 = C2.mean(axis=0)
    
    if Model == 1: model = IndependentCovar()
    elif Model == 2: model = EqualCovar()
    else: model = DiagonalCovar()
    
    S1, S2 = model.getCovar(C1, C2, P_C1, P_C2, m1, m2)
    test = pd.read_csv(testing_data, header=None)
    error = testError(model, test, P_C1, P_C2, m1, m2, S1, S2)
    return pd.DataFrame({
        "P(C1)": P_C1, "P(C2)": P_C2,
        "m1": [m1], "m2": [m2],
        "S1": [S1], "S2": [S2],
        "Test error": error,
    })

##### Calculation of parameters and error rates for all combinations of models and datasets

In [4]:
data_files = [("training_data1.txt", "test_data1.txt"),
              ("training_data2.txt", "test_data2.txt"),
              ("training_data3.txt", "test_data3.txt")]

results = []

for file in data_files:
    file_results = []
    for Model in range(1, 4):
        file_results.append(MultiGaussian(file[0], file[1], Model))
    results.append(file_results)

##### Printing out parameters

In [5]:
np.set_printoptions(precision=2)
for i in range(3):
    print(f'\nModel {i+1}:')
    output = results[0][i]
    print("  P(C1):", output["P(C1)"][0])
    print("  P(C2):", output["P(C2)"][0])
    print("  m1:", output["m1"][0])
    print("  m2:", output["m2"][0])
    print("  S1:", str(output["S1"][0]).replace('\n', '\n'))
    print("  S2:", str(output["S2"][0]).replace('\n', '\n'))


Model 1:
  P(C1): 0.3
  P(C2): 0.7
  m1: [ 0.43  2.02  3.18 -2.43 -2.52  3.24 -5.52 -6.69]
  m2: [ 4.58  6.49  6.43  1.69  2.29  8.36 -0.17 -1.8 ]
  S1: [[ 1.86  0.23  0.75  1.    0.42  1.22  1.13 -1.19]
       [ 0.23  3.54  0.3  -0.13  1.53  1.   -0.19  3.17]
       [ 0.75  0.3   7.84  1.29 -0.41  1.72  0.34  0.22]
       [ 1.   -0.13  1.29  4.09  0.92  0.72  1.03  1.91]
       [ 0.42  1.53 -0.41  0.92  4.    0.97 -0.53  3.32]
       [ 1.22  1.    1.72  0.72  0.97  3.93 -0.19  2.22]
       [ 1.13 -0.19  0.34  1.03 -0.53 -0.19  4.08 -1.65]
       [-1.19  3.17  0.22  1.91  3.32  2.22 -1.65 16.53]]
  S2: [[ 3.42  2.07  2.57  2.61  1.77  1.83  2.68  2.93]
       [ 2.07  5.78  2.18  2.72  3.16  2.89  2.76  5.85]
       [ 2.57  2.18  8.71  3.38  2.83  2.23  2.75  5.18]
       [ 2.61  2.72  3.38  8.17  3.58  2.66  2.02  8.4 ]
       [ 1.77  3.16  2.83  3.58  5.57  2.91  3.25  4.82]
       [ 1.83  2.89  2.23  2.66  2.91  3.73  2.23  4.48]
       [ 2.68  2.76  2.75  2.02  3.25  2.23  8.21  4.

#### b.2 Prediction accuracy of the learning method

Error rate of each dataset with each model

In [6]:
print("Error rates:", end='')
for i in range(3):
    print(f'\n  For dataset {i+1}:')
    for j in range(3):
        print(f'    M{j+1}: {results[i][j]["Test error"][0]}')

Error rates:
  For dataset 1:
    M1: 0.22
    M2: 0.17
    M3: 0.18

  For dataset 2:
    M1: 0.23
    M2: 0.55
    M3: 0.52

  For dataset 3:
    M1: 0.11
    M2: 0.45
    M3: 0.06


At the end we can conclude that : 

> For dataset 1 : Model 2 (Equal Covariances)

> For dataset 2 : Model 1 (Independent Covariances)

> For dataset 3 : Model 3 (Diagonal Covariances )

<br>
End of Topic-B